In [1]:
import numpy as np
from pulp import *

In [29]:
def dx_ocs(gamma, kmax):
    x = np.array([1 - pow(0.5, k)*pow(1-gamma, max(0,k-1)) for k in range(kmax+2)])
    return x[1:] - x[:kmax+1]

In [2]:
def ratio_lp(gamma, kmax, kappa=1.5):
    #dx = dx_ocs(gamma, kmax)
    #print('dx:', dx)
    
    lp = LpProblem('ratio lp', LpMaximize)
    
    Gamma = LpVariable('Γ', 0.)
    lp += Gamma
    
    a = LpVariable.dict('a', range(kmax+1), 0.)
    b = LpVariable.dict('b', range(kmax+1), 0.)
    
    # gain split in deterministic rounds
    for k in range(kmax+1):
        #lp += sum([a[i] for i in range(k, kmax+1)]) + kappa*b[k] <= sum(dx[k:])
        lp += sum([a[i] for i in range(k, kmax+1)]) + kappa*b[k] <= pow(0.5, k)*pow(1-gamma, max(k-1, 0))
    
    # gain split in randomized rounds
    lp += a[0] + b[0] <= 0.5
    for k in range(1, kmax+1):
        #lp += a[k] + b[k] <= dx[k]
        lp += a[k] + b[k] <= pow(0.5, k+1)*pow(1-gamma, k-1)*(1+gamma)
    
    # compensation
    lp += a[0] >= gamma/2
    
    # boundary condition
    lp += sum([a[i] for i in range(kmax+1)]) >= Gamma
    
    # approximate dual feasibility: randomized round, not chosen
    for k in range(kmax+1):
        lp += sum([a[i] for i in range(k)]) + 2*b[k] >= Gamma
    
    # approximate dual feasibility: randomized round, chosen
    for k in range(kmax):
        lp += sum([a[i] for i in range(k+1)]) + kappa*b[k] >= Gamma

    lp.solve()
    
    return (value(Gamma),
            [value(a[k]) for k in range(kmax+1)],
            [value(b[k]) for k in range(kmax+1)])

In [3]:
Gamma, a, b = ratio_lp(1/16, 8)

/usr/local/lib/python3.7/site-packages/pulp/pulp.py:1114: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [4]:
Gamma

0.50503489

In [5]:
a

[0.24748256,
 0.13684883,
 0.06415997,
 0.030093105,
 0.014133321,
 0.0066657617,
 0.0031857269,
 0.0015850361,
 0.00088057561]

In [6]:
b

[0.25251744,
 0.12877617,
 0.060351748,
 0.028271763,
 0.013225211,
 0.0061585501,
 0.0028256693,
 0.0012328059,
 0.00044028781]

In [9]:
for kappa in np.arange(1.0, 2.0 + 1/16, 1/16):
    Gamma, a, b = ratio_lp(1/16, 8, kappa)
    print('kappa = %f, Gamma = %f' % (kappa, Gamma))

kappa = 1.000000, Gamma = 0.500000
kappa = 1.062500, Gamma = 0.505035
kappa = 1.125000, Gamma = 0.505035
kappa = 1.187500, Gamma = 0.505035
kappa = 1.250000, Gamma = 0.505035
kappa = 1.312500, Gamma = 0.505035
kappa = 1.375000, Gamma = 0.505035
kappa = 1.437500, Gamma = 0.505035
kappa = 1.500000, Gamma = 0.505035
kappa = 1.562500, Gamma = 0.505035
kappa = 1.625000, Gamma = 0.505035
kappa = 1.687500, Gamma = 0.505035
kappa = 1.750000, Gamma = 0.505035
kappa = 1.812500, Gamma = 0.505035
kappa = 1.875000, Gamma = 0.505035
kappa = 1.937500, Gamma = 0.502645
kappa = 2.000000, Gamma = 0.500000


In [98]:
Gamma, a, b = ratio_lp(1/(3*np.sqrt(3)), 8)

In [99]:
Gamma

0.51462476

In [100]:
a

[0.24268762,
 0.16214395,
 0.065473776,
 0.026442834,
 0.0106862,
 0.0043286876,
 0.0017686174,
 0.00074532606,
 0.00034774797]

In [101]:
b

[0.25731238,
 0.13596857,
 0.054896594,
 0.022159706,
 0.0089382893,
 0.0035951895,
 0.0014308457,
 0.00054653701,
 0.00017387398]

In [102]:
(13*np.sqrt(13)-35)/108

0.10992746834288755

In [134]:
Gamma, a, b = ratio_lp(0.109927, 8)

In [135]:
Gamma

0.50867279

In [136]:
a

[0.24566361,
 0.14597716,
 0.064973493,
 0.028928071,
 0.01289279,
 0.0057658754,
 0.0026081951,
 0.0012239971,
 0.00063960608]

In [137]:
b

[0.25433639,
 0.13150459,
 0.058516014,
 0.026029267,
 0.011565232,
 0.0051188368,
 0.0022358991,
 0.00093180161,
 0.00031980304]

In [14]:
kmax = 40
Gamma, a, b = ratio_lp(1/(3*np.sqrt(3)), kmax)
dx = dx_ocs(1/(3*np.sqrt(3)), kmax)

In [15]:
a / dx

array([0.48537024, 0.54388926, 0.54388927, 0.54388928, 0.54388928,
       0.54388927, 0.54388922, 0.5438891 , 0.54388863, 0.54388692,
       0.54388049, 0.54385663, 0.54376802, 0.54343882, 0.54221586,
       0.53767256, 0.5207945 , 0.67944492, 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 2.07695688, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [16]:
b / dx

array([0.51462976, 0.45611073, 0.45611073, 0.45611072, 0.45611073,
       0.45611074, 0.45611078, 0.45611089, 0.45611136, 0.45611309,
       0.45611951, 0.45614337, 0.45623198, 0.45656119, 0.45778417,
       0.46232744, 0.47920552, 0.26957837, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.55906352, 0.        , 0.        , 0.        , 0.        ,
       1.11804256, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [17]:
Gamma, a, b = ratio_lp(0.10993, 8)

In [18]:
Gamma

0.50866813

In [36]:
def unweighted_lp(kmax):
    g = np.ones(kmax+2)
    for i in range(2,kmax+2):
        g[i] = g[i-1] - (13*np.sqrt(13)-35)/108*g[i-2]
        #g[i] = g[i-1] - g[i-2]/16
    x = np.array([1-2**(-k)*g[k] for k in range(kmax+2)])
    print(x)
    
    lp = LpProblem('ratio lp', LpMaximize)
    
    Gamma = LpVariable('Γ', 0., 1.)
    lp += Gamma
    
    a = LpVariable.dict('a', range(kmax+1), 0.)
    b = LpVariable.dict('b', range(kmax+1), 0.)
    
    for k in range(kmax+1):
        lp += a[k] + b[k] <= x[k+1] - x[k]
        lp += sum([a[i] for i in range(k)]) + 2*b[k] >= Gamma
        if k < kmax:
            lp += b[k] >= b[k+1]
    lp += sum([a[i] for i in range(kmax+1)]) >= Gamma

    lp.solve()
    
    return (value(Gamma),
            [value(a[k]) for k in range(kmax+1)],
            [value(b[k]) for k in range(kmax+1)])

In [38]:
unweighted_lp(8)

[0.         0.5        0.77748187 0.90248187 0.95735615 0.98135805
 0.99185096 0.9964378  0.99844285 0.99931932]


(0.50898643,
 [0.24550678,
  0.14574204,
  0.066131198,
  0.029071078,
  0.012734243,
  0.0055923636,
  0.0024824754,
  0.0011419306,
  0.00058431452],
 [0.25449322,
  0.13173982,
  0.058868802,
  0.025803202,
  0.011267664,
  0.0049005421,
  0.0021043603,
  0.00086312257,
  0.00029215726])

In [40]:
g = np.ones(10)
for i in range(2,10):
    g[i] = g[i-1] - (13*np.sqrt(13)-35)/108*g[i-2]
print(g)

[1.         1.         0.89007253 0.78014506 0.68230164 0.59654227
 0.52153858 0.4559622  0.39863078 0.34850801]
